In [ ]:
pip install Pillow

In [ ]:
# delete files in a directory
directory_path = "/content/dataset_augmented/food"
for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        if os.path.isfile(item_path):
            os.remove(item_path)  # Remove the file
            print(f"Removed file: {item_path}")
        else:
            print(f"Skipping directory: {item_path}")

In [ ]:
# number of itmes in a directory
src =  "/content/dataset/humans"
all_items = os.listdir(src)
len([item for item in all_items if os.path.isfile(os.path.join(src, item))])

5000

In [ ]:
# transfer a set number of fiels from one directrory to another

files_transferred = 0
import shutil
src = "/content/Fast Food Classification V2/Train/Hot Dog"
dst = "/content/dataset/food"
for file in os.listdir(src):
        if files_transferred < 1000:
            source_file_path = os.path.join(src, file)
            destination_file_path = os.path.join(dst, file)

            # Ensure we're only transferring files (not directories)
            if os.path.isfile(source_file_path):
                shutil.move(source_file_path, destination_file_path)
                print(f"Transferred: {file}")
                files_transferred += 1
        else:
            break  # Stop transferring if we've reached the specified number of files

print("Transferred")


### Augmenting dataset

In [ ]:
import random
from PIL import Image, ImageFilter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import albumentations as A
import random
import torch

In [ ]:
def augment(input_diretory_path):

  def pixelate(input_image_path, pixel_size=5):
    with Image.open(input_image_path) as image:
        # Calculate the size of the pixelated image, reducing its resolution
        small_image = image.resize(
            (image.width // pixel_size, image.height // pixel_size),
            resample=Image.BILINEAR
        )

        # Scale it back to the original size
        result_image = small_image.resize(
            image.size,
            Image.NEAREST
        )
        path_without_extension, _ = os.path.splitext(input_image_path)
        output_image_path = path_without_extension + "_pixelated.jpg"
        # Save the pixelated image
        result_image.save(output_image_path)

  def add_grain(input_image_path, grain_level=5):
    # Open the image
    with Image.open(input_image_path) as image:
        # Convert the image into a numpy array
        image_array = np.array(image)

        # Generate noise array, scaled by the grain level
        noise = np.random.randint(-grain_level, grain_level, image_array.shape, dtype='int16')

        # Add noise to the image array, ensuring values stay in the [0, 255] range
        noisy_image_array = image_array.astype('int16') + noise
        noisy_image_array = np.clip(noisy_image_array, 0, 255)

        # Convert the noisy image array back into an image
        noisy_image = Image.fromarray(noisy_image_array.astype('uint8'))
        path_without_extension, _ = os.path.splitext(input_image_path)
        output_image_path = path_without_extension + "_grainy.jpg"
        # Save the grainy image
        noisy_image.save(output_image_path)

  def blur_image(input_image_path, blur_radius=5):
    with Image.open(input_image_path) as image:
        # Apply the blur filter with the specified blur radius
        blurred_image = image.filter(ImageFilter.GaussianBlur(blur_radius))

        path_without_extension, _ = os.path.splitext(input_image_path)
        output_image_path = path_without_extension + "_blurred.jpg"

        # Save the blurred image
        blurred_image.save(output_image_path)

  def motion_blur(image_path, intensity=10, angle=30):
    image = cv2.imread(image_path)
    # Generating the motion blur kernel
    kernel_size = intensity
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    kernel = cv2.warpAffine(kernel, cv2.getRotationMatrix2D((kernel_size / 2 - 0.5, kernel_size / 2 - 0.5), angle, 1.0), (kernel_size, kernel_size))
    kernel = kernel / kernel_size

    # Applying the kernel to the input image
    blurred_image = cv2.filter2D(image, -1, kernel)

    return blurred_image

    # New Step: Define output directory
    output_directory = '/content/'

    # Example usage with BMP images and saving to a separate directory
    image_path = '/content/drive/MyDrive/Classes/566/Project/rider-10.jpg'
    blurred_image = apply_motion_blur(image_path, intensity=10, angle=30)
    path_without_extension, _ = os.path.splitext(input_image_path)
    output_image_path = path_without_extension + "_motion_blurred.jpg"
    cv2.imwrite(output_image_path, blurred_image)  # Save the blurred image in the specified output directory


  def lens_flare(image_path):
    def seed_everything(seed):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

    def visualize(image):
        plt.figure(figsize=(10, 10))
        plt.axis('off')
        plt.imshow(image)

    def visualize_multiple(nrows, ncols, img, transform):
        fig, axes = plt.subplots(nrows,ncols)
        fig.set_figheight(15)
        fig.set_figwidth(15)
        num_iter = 0
        for row in range(nrows):
            for col in range(ncols):
                augmented_img = transform[num_iter](image=img)['image']
                axes[row,col].imshow(augmented_img)
                axes[row,col].grid(False)
                axes[row,col].set_xticks([])
                axes[row,col].set_yticks([])
                num_iter += 1
        return fig, axes

    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    transform = [A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5),
                                  angle_lower=0, angle_upper=1,
                                  num_flare_circles_lower=6,
                                  num_flare_circles_upper=10,
                                  src_radius=400, src_color=(255, 255, 255),
                                  always_apply=True) for _ in range(9)]
    path_without_extension, _ = os.path.splitext(input_image_path)
    output_image_path = path_without_extension + "_motion_blurred.jpg"

    # Apply the first transformation in the list to the image
    augmented_img = transform[random.randint(0, len(transform))-1](image=img)['image']

    # Save the transformed image as a .jpg file
    cv2.imwrite(output_image_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))


  num_of_files = 0
  num_of_files_limit = 200
  for filename in os.listdir(input_diretory_path):
    if(num_of_files < num_of_files_limit):
      image_path = os.path.join(input_images_path, filename)
      pixelate(input_image_path, pixel_size=5)
      add_grain(input_image_path, grain_level=30)
      blur_image(input_image_path, blur_radius=2)
      motion_blur(input_image_path)
      lens_flare(input_image_path)
      num_of_files+=1


In [ ]:
augment("/content/dataset_augmented/food")

In [ ]:
!cp -r '/content/dataset/' '/content/drive/MyDrive/'